In [15]:
import fnmatch, os, sys
# sys.path.append('/Users/sherryan/Desktop/cage_data/')
# print(sys.path)
import cage_data
import numpy as np
import pickle as pickle

In [16]:
# data_path = '/Users/sherryan/ssa-pop/'
curr_dir = os.getcwd()
data_path = "%s/pickle_files/" % curr_dir
# files_name = ['Pop_20210709_Cage_001.pkl']
files_name = ["Pop_20210709_Cage_004.pkl"]
cage_data_list = []
for file in files_name:
    print('The file %s is going to be loaded'%(data_path + file))
    with open ( data_path + file, 'rb' ) as fp:
        cage_data = pickle.load(fp)
        cage_data_list.append(cage_data)


The file /Users/andrewshen/Github_Repos/neural_decoding/data/pickle_files/Pop_20210709_Cage_004.pkl is going to be loaded


In [17]:
# Andrew's code
my_cage_data = cage_data_list[0]

In [18]:
my_cage_data.pre_processing_summary()
# We see that the spikes are smoothed

This is a non-sorted file
EMG filtered? -- False
EMG filtered? -- False
Cortical data cleaned? -- False
Data binned? -- True
Spikes smoothed? -- True


In [19]:
bin_size = .025

In [20]:
keys = cage_data.behave_event.keys()
keys

dict_keys(['bar_touch', 'treat_touch', 'pg_force_onset'])

In [21]:

def find_start_end_event(timing, cage_data):
    timeframe = cage_data.binned['timeframe']
    segment_range = np.where((timeframe>=timing-0.5) & (timeframe<=timing+2))[0] # Changed for expanded time range
    start_idx = segment_range[0]
    end_idx = segment_range[-1]
    return start_idx, end_idx


In [22]:
behav_dict = {'crawl': [], 'precision': [], 'power': []}

In [23]:
# Andrew's code

# To get the binned spike counts
binned_spike_counts = my_cage_data.binned['spikes']

# To get the rectified, filtered and downsampled EMGs
filtered_EMG = my_cage_data.binned['filtered_EMG']

# To get the time frame of the binned data
timeframe = my_cage_data.binned['timeframe']

m1_data = np.transpose(binned_spike_counts)
emg_data = np.transpose(filtered_EMG)

In [24]:
trial_length = None # For splitting into trials at the end

concat_spikes = []
curr_idx = 0
for key in keys:  
    if key == 'bar_touch':
        event = 'crawl'
    if key == 'treat_touch':
        event = 'precision'
    if key == 'pg_force_onset':
        event = 'power'
    behav_dict[event].append(curr_idx)
    for cage_data in cage_data_list:
        spikes = np.array(cage_data.binned['spikes'])
        for timing in cage_data.behave_event[key]:
            start_idx, end_idx = find_start_end_event(timing, cage_data)
            trial_length = end_idx-start_idx+1
            curr_idx += (end_idx - start_idx + 1)
            concat_spikes.append(spikes[:,start_idx:end_idx+1])
            behav_dict[event].append(curr_idx)



In [25]:
# Andrew's code to keep only timestamps Set2 labeled behaviors for M1 and EMG

m1_data_labeled_set2 = []
emg_data_labeled_set2 = []
behavioral_labels_set2 = []

separate_modes = False
trial_format = True
num_bins = 1

m1_data_labeled_set2_crawl = []
emg_data_labeled_set2_crawl = []
behavioral_labels_set2_crawl = []
m1_data_labeled_set2_precision = []
emg_data_labeled_set2_precision = []
behavioral_labels_set2_precision = []
m1_data_labeled_set2_power = []
emg_data_labeled_set2_power = []
behavioral_labels_set2_power = []

concat_spikes = []
curr_idx = 0
for key in keys:  
    if key == 'bar_touch':
        event = 'crawl'
    if key == 'treat_touch':
        event = 'precision'
    if key == 'pg_force_onset':
        event = 'power'

    behav_dict[event].append(curr_idx)
    for cage_data in cage_data_list:
        spikes = np.array(cage_data.binned['spikes'])
        for timing in cage_data.behave_event[key]:
            start_idx, end_idx = find_start_end_event(timing, cage_data)
            curr_idx += (end_idx - start_idx + 1)
            concat_spikes.append(spikes[:,start_idx:end_idx+1])

            # Andrew's code
            behavioral_label = event

            # Add code to save data for separate modes
            if separate_modes == False:

                # Code to save timestamps without being split into trials
                for i in range(start_idx, end_idx+1):
                    if len(m1_data_labeled_set2) == 0:
                        m1_data_labeled_set2 = np.array(np.expand_dims(np.concatenate(m1_data[i+1-num_bins:i+1]), axis=0))
                        emg_data_labeled_set2 = np.array(np.expand_dims(np.concatenate(emg_data[i+1-num_bins:i+1]), axis=0))
                        behavioral_labels_set2 = np.array([behavioral_label])
                    else:
                        m1_data_labeled_set2 = np.concatenate((m1_data_labeled_set2, np.expand_dims(np.concatenate(m1_data[i+1-num_bins:i+1]), axis=0)))
                        emg_data_labeled_set2 = np.concatenate((emg_data_labeled_set2, np.expand_dims(np.concatenate(emg_data[i+1-num_bins:i+1]), axis=0)))
                        behavioral_labels_set2 = np.concatenate((behavioral_labels_set2, [behavioral_label]))

                # Original code to save only timestamps with behavioral labels (new code should work with any bin size)
                # if len(m1_data_labeled_set2) == 0:
                #     m1_data_labeled_set2 = m1_data[start_idx:end_idx+1]
                #     emg_data_labeled_set2 = emg_data[start_idx:end_idx+1]
                #     behavioral_labels_set2 = np.array([behavioral_label for i in range(end_idx-start_idx+1)])
                # else:
                #     m1_data_labeled_set2 = np.concatenate((m1_data_labeled_set2, m1_data[start_idx:end_idx+1]))
                #     emg_data_labeled_set2 = np.concatenate((emg_data_labeled_set2, emg_data[start_idx:end_idx+1]))
                #     behavioral_labels_set2 = np.concatenate((behavioral_labels_set2, np.array([behavioral_label for i in range(end_idx-start_idx+1)])))

            elif separate_modes == True:
                if behavioral_label == "crawl":
                    for i in range(start_idx, end_idx+1):
                        if len(m1_data_labeled_set2_crawl) == 0:
                            m1_data_labeled_set2_crawl = np.array(np.expand_dims(np.concatenate(m1_data[i+1-num_bins:i+1]), axis=0))
                            emg_data_labeled_set2_crawl = np.array(np.expand_dims(np.concatenate(emg_data[i+1-num_bins:i+1]), axis=0))
                            behavioral_labels_set2_crawl = np.array([behavioral_label])
                        else:
                            m1_data_labeled_set2_crawl = np.concatenate((m1_data_labeled_set2_crawl, np.expand_dims(np.concatenate(m1_data[i+1-num_bins:i+1]), axis=0)))
                            emg_data_labeled_set2_crawl = np.concatenate((emg_data_labeled_set2_crawl, np.expand_dims(np.concatenate(emg_data[i+1-num_bins:i+1]), axis=0)))
                            behavioral_labels_set2_crawl = np.concatenate((behavioral_labels_set2_crawl, [behavioral_label]))
                
                elif behavioral_label == "precision":
                    for i in range(start_idx, end_idx+1):
                        if len(m1_data_labeled_set2_precision) == 0:
                            m1_data_labeled_set2_precision = np.array(np.expand_dims(np.concatenate(m1_data[i+1-num_bins:i+1]), axis=0))
                            emg_data_labeled_set2_precision = np.array(np.expand_dims(np.concatenate(emg_data[i+1-num_bins:i+1]), axis=0))
                            behavioral_labels_set2_precision = np.array([behavioral_label])
                        else:
                            m1_data_labeled_set2_precision = np.concatenate((m1_data_labeled_set2_precision, np.expand_dims(np.concatenate(m1_data[i+1-num_bins:i+1]), axis=0)))
                            emg_data_labeled_set2_precision = np.concatenate((emg_data_labeled_set2_precision, np.expand_dims(np.concatenate(emg_data[i+1-num_bins:i+1]), axis=0)))
                            behavioral_labels_set2_precision = np.concatenate((behavioral_labels_set2_precision, [behavioral_label]))

                if behavioral_label == "power":
                    for i in range(start_idx, end_idx+1):
                        if len(m1_data_labeled_set2_power) == 0:
                            m1_data_labeled_set2_power = np.array(np.expand_dims(np.concatenate(m1_data[i+1-num_bins:i+1]), axis=0))
                            emg_data_labeled_set2_power = np.array(np.expand_dims(np.concatenate(emg_data[i+1-num_bins:i+1]), axis=0))
                            behavioral_labels_set2_power = np.array([behavioral_label])
                        else:
                            m1_data_labeled_set2_power = np.concatenate((m1_data_labeled_set2_power, np.expand_dims(np.concatenate(m1_data[i+1-num_bins:i+1]), axis=0)))
                            emg_data_labeled_set2_power = np.concatenate((emg_data_labeled_set2_power, np.expand_dims(np.concatenate(emg_data[i+1-num_bins:i+1]), axis=0)))
                            behavioral_labels_set2_power = np.concatenate((behavioral_labels_set2_power, [behavioral_label]))

                # Original code
                # if behavioral_label == "crawl":
                #     if len(m1_data_labeled_set2_crawl) == 0:
                #         m1_data_labeled_set2_crawl = m1_data[start_idx:end_idx+1]
                #         emg_data_labeled_set2_crawl = emg_data[start_idx:end_idx+1]
                #         behavioral_labels_set2_crawl = np.array([behavioral_label for i in range(end_idx-start_idx+1)])
                #     else:
                #         m1_data_labeled_set2_crawl = np.concatenate((m1_data_labeled_set2_crawl, m1_data[start_idx:end_idx+1]))
                #         emg_data_labeled_set2_crawl = np.concatenate((emg_data_labeled_set2_crawl, emg_data[start_idx:end_idx+1]))
                #         behavioral_labels_set2_crawl = np.concatenate((behavioral_labels_set2_crawl, np.array([behavioral_label for i in range(end_idx-start_idx+1)])))
                # elif behavioral_label == "precision":
                #     if len(m1_data_labeled_set2_precision) == 0:
                #         m1_data_labeled_set2_precision = m1_data[start_idx:end_idx+1]
                #         emg_data_labeled_set2_precision = emg_data[start_idx:end_idx+1]
                #         behavioral_labels_set2_precision = np.array([behavioral_label for i in range(end_idx-start_idx+1)])
                #     else:
                #         m1_data_labeled_set2_precision = np.concatenate((m1_data_labeled_set2_precision, m1_data[start_idx:end_idx+1]))
                #         emg_data_labeled_set2_precision = np.concatenate((emg_data_labeled_set2_precision, emg_data[start_idx:end_idx+1]))
                #         behavioral_labels_set2_precision = np.concatenate((behavioral_labels_set2_precision, np.array([behavioral_label for i in range(end_idx-start_idx+1)])))
                # elif behavioral_label == "power":
                #     if len(m1_data_labeled_set2_power) == 0:
                #         m1_data_labeled_set2_power = m1_data[start_idx:end_idx+1]
                #         emg_data_labeled_set2_power = emg_data[start_idx:end_idx+1]
                #         behavioral_labels_set2_power = np.array([behavioral_label for i in range(end_idx-start_idx+1)])
                #     else:
                #         m1_data_labeled_set2_power = np.concatenate((m1_data_labeled_set2_power, m1_data[start_idx:end_idx+1]))
                #         emg_data_labeled_set2_power = np.concatenate((emg_data_labeled_set2_power, emg_data[start_idx:end_idx+1]))
                #         behavioral_labels_set2_power = np.concatenate((behavioral_labels_set2_power, np.array([behavioral_label for i in range(end_idx-start_idx+1)])))

# Save M1, EMG, and Set2 behavioral labels
out_path = "/Users/andrewshen/Desktop/neural_decoding/data/"
if separate_modes == False:
    # Format into trials
    if trial_format:
        m1_data_labeled_set2 = np.reshape(m1_data_labeled_set2, (m1_data_labeled_set2.shape[0]//(trial_length), trial_length, m1_data_labeled_set2.shape[1]))
        emg_data_labeled_set2 = np.reshape(emg_data_labeled_set2, (emg_data_labeled_set2.shape[0]//(trial_length), trial_length, emg_data_labeled_set2.shape[1]))
        behavioral_labels_set2 = np.reshape(behavioral_labels_set2, (behavioral_labels_set2.shape[0]//(trial_length), trial_length))
    # np.save(out_path + "set2_data/m1_set2_t100_b10", m1_data_labeled_set2)
    # np.save(out_path + "set2_data/emg_set2_t100_b10", emg_data_labeled_set2)
    # np.save(out_path + "set2_data/behavioral_set2_t100_b10", behavioral_labels_set2)

    print(m1_data_labeled_set2.shape)
    print(emg_data_labeled_set2.shape)
    print(behavioral_labels_set2.shape)

elif separate_modes == True:

    # Format into trials
    if trial_format:
        m1_data_labeled_set2_crawl = np.reshape(m1_data_labeled_set2_crawl, (m1_data_labeled_set2_crawl.shape[0]//(trial_length), trial_length, m1_data_labeled_set2_crawl.shape[1]))
        emg_data_labeled_set2_crawl = np.reshape(emg_data_labeled_set2_crawl, (emg_data_labeled_set2_crawl.shape[0]//(trial_length), trial_length, emg_data_labeled_set2_crawl.shape[1]))
        behavioral_labels_set2_crawl = np.reshape(behavioral_labels_set2_crawl, (behavioral_labels_set2_crawl.shape[0]//(trial_length), trial_length))
        m1_data_labeled_set2_precision = np.reshape(m1_data_labeled_set2_precision, (m1_data_labeled_set2_precision.shape[0]//(trial_length), trial_length, m1_data_labeled_set2_precision.shape[1]))
        emg_data_labeled_set2_precision = np.reshape(emg_data_labeled_set2_precision, (emg_data_labeled_set2_precision.shape[0]//(trial_length), trial_length, emg_data_labeled_set2_precision.shape[1]))
        behavioral_labels_set2_precision = np.reshape(behavioral_labels_set2_precision, (behavioral_labels_set2_precision.shape[0]//(trial_length), trial_length))
        m1_data_labeled_set2_power = np.reshape(m1_data_labeled_set2_power, (m1_data_labeled_set2_power.shape[0]//(trial_length), trial_length, m1_data_labeled_set2_power.shape[1]))
        emg_data_labeled_set2_power = np.reshape(emg_data_labeled_set2_power, (emg_data_labeled_set2_power.shape[0]//(trial_length), trial_length, emg_data_labeled_set2_power.shape[1]))
        behavioral_labels_set2_power = np.reshape(behavioral_labels_set2_power, (behavioral_labels_set2_power.shape[0]//(trial_length), trial_length))
    # np.save(out_path + "set2_data/sep_modes_b10/m1_set2_t100_b10_crawl", m1_data_labeled_set2_crawl)
    # np.save(out_path + "set2_data/sep_modes_b10/emg_set2_t100_b10_crawl", emg_data_labeled_set2_crawl)
    # np.save(out_path + "set2_data/sep_modes_b10/behavioral_set2_t100_b10_crawl", behavioral_labels_set2_crawl)
    # np.save(out_path + "set2_data/sep_modes_b10/m1_set2_t100_b10_precision", m1_data_labeled_set2_precision)
    # np.save(out_path + "set2_data/sep_modes_b10/emg_set2_t100_b10_precision", emg_data_labeled_set2_precision)
    # np.save(out_path + "set2_data/sep_modes_b10/behavioral_set2_t100_b10_precision", behavioral_labels_set2_precision)
    # np.save(out_path + "set2_data/sep_modes_b10/m1_set2_t100_b10_power", m1_data_labeled_set2_power)
    # np.save(out_path + "set2_data/sep_modes_b10/emg_set2_t100_b10_power", emg_data_labeled_set2_power)
    # np.save(out_path + "set2_data/sep_modes_b10/behavioral_set2_t100_b10_power", behavioral_labels_set2_power)

    print(m1_data_labeled_set2_crawl.shape)
    print(emg_data_labeled_set2_crawl.shape)
    print(behavioral_labels_set2_crawl.shape)
    print(m1_data_labeled_set2_precision.shape)
    print(emg_data_labeled_set2_precision.shape)
    print(behavioral_labels_set2_precision.shape)
    print(m1_data_labeled_set2_power.shape)
    print(emg_data_labeled_set2_power.shape)
    print(behavioral_labels_set2_power.shape)

(98, 100, 95)
(98, 100, 16)
(98, 100)


In [ ]:
out_path

In [ ]:
behav_dict